# 基于streamlit的大模型应用开发2

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
model_ty = Tongyi(temperature=0)
model_qf = QianfanLLMEndpoint(model="ERNIE-Bot")
chat_qf = QianfanChatEndpoint(model="ERNIE-Bot")
chat_xh = ChatSparkLLM()
chat_ty= ChatTongyi()

In [4]:
import langchain
langchain.__version__

'0.1.16'

In [5]:
#! pip install --upgrade langchain -i https://mirrors.aliyun.com/pypi/simple

## 基于streamlit的大模型问答应用实现-通义千问

* 原生通义千问问答实现

* 基于streamlit回话状态实现记忆功能

In [3]:
import dashscope
from dashscope import Generation
import streamlit as st

dashscope.api_key = ""

In [4]:
prompt=[]
def get_response(mess):
    response = Generation.call(
        model='qwen-turbo',
        messages=mess,
        result_format='message', # 将输出设置为message形式
    )
    return response

def prompt_yun(ct):
    prompt.append({"role":"user","content":ct})
    return prompt

In [5]:
p_y=prompt_yun("你好")
res=get_response(p_y)

In [6]:
res

GenerationResponse(status_code=<HTTPStatus.OK: 200>, request_id='c3470d10-f056-93c6-96e3-92bef9f33053', code='', message='', output=GenerationOutput(text=None, choices=[Choice(finish_reason='stop', message=Message({'role': 'assistant', 'content': '你好！很高兴能为你服务。有什么可以帮助你的吗？'}))], finish_reason=None), usage=GenerationUsage(input_tokens=9, output_tokens=12))

In [9]:
dict(res.output.choices[0].message)

{'role': 'assistant', 'content': '你好！很高兴能为你服务。有什么可以帮助你的吗？'}

In [7]:
res.output.choices[0].message['content']

'你好！很高兴能为你服务。有什么可以帮助你的吗？'

## 参数的输入-langchain-streamlit大模型应用开发

* streamlit文档

https://docs.streamlit.io/develop/api-reference

* st.sidebar.text_input

* st.sidebar.select_slider

In [10]:
help(Tongyi)

Help on class Tongyi in module langchain_community.llms.tongyi:

class Tongyi(langchain_core.language_models.llms.BaseLLM)
 |  Tongyi(*, name: Optional[str] = None, cache: ForwardRef('Union[BaseCache, bool, None]') = None, verbose: bool = None, callbacks: ForwardRef('Callbacks') = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, custom_get_token_ids: Optional[Callable[[str], List[int]]] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, client: Any = None, model_name: str = 'qwen-plus', model_kwargs: Dict[str, Any] = None, top_p: float = 0.8, dashscope_api_key: Optional[str] = None, streaming: bool = False, max_retries: int = 10) -> None
 |  
 |  Tongyi Qwen large language models.
 |  
 |  To use, you should have the ``dashscope`` python package installed, and the
 |  environment variable ``DASHSCOPE_API_KEY`` set with your API key, or pass
 |  it as a named parameter to the constructor.
 |  
 |  Example:
 |  

## RAG应用实现

https://blog.csdn.net/itnerd/article/details/137522411

https://blog.csdn.net/csdn_xmj/article/details/136592846

https://github.com/langchain-ai/streamlit-agent/blob/main/streamlit_agent/chat_with_documents.py

https://docs.streamlit.io/develop/api-reference/widgets/st.file_uploader

* 原始实现

In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
from langchain.vectorstores import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

model_qf = QianfanLLMEndpoint(temperature=0.1)
loader_txt = TextLoader(r'G:\云岚宗.txt', encoding='utf8')
docs_txt = loader_txt.load()
text_splitter_txt = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
documents_txt = text_splitter_txt.split_documents(docs_txt)
embeddings_qf = QianfanEmbeddingsEndpoint()
vectordb = Chroma.from_documents(documents=documents_txt, embedding=embeddings_qf)

prompt = ChatPromptTemplate.from_template("""使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话，你的回答中不可以带有星号。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
<context>
{context}
</context>

Question: {input}""")
#创建检索链

retriever = vectordb.as_retriever()
memory = ConversationSummaryMemory(
    llm=model_qf, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm=model_qf, retriever=retriever, memory=memory)

[INFO] [05-14 09:12:48] openapi_requestor.py:316 [t:14188]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [05-14 09:12:48] oauth.py:207 [t:14188]: trying to refresh access_token for ak `og6mWr***`
[INFO] [05-14 09:12:48] oauth.py:220 [t:14188]: sucessfully refresh access_token


In [9]:
memory.chat_memory.messages

[]

In [10]:
res = qa.invoke(
    {"question": "萧炎是谁"}
)
print(res["answer"])

[WARNING] [05-14 09:13:00] base.py:495 [t:14188]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-14 09:13:00] openapi_requestor.py:316 [t:14188]: requesting llm api endpoint: /chat/eb-instant
[INFO] [05-14 09:13:01] openapi_requestor.py:316 [t:14188]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [05-14 09:13:01] base.py:495 [t:14188]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-14 09:13:01] openapi_requestor.py:316 [t:14188]: requesting llm api endpoint: /chat/eb-instant
[WARNING] [05-14 09:13:03] base.py:495 [t:14188]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-14 09:13:03] openapi_requestor.py:316 [t:14188]: requesting llm api endpoint: /chat/eb-instant


From the context, Xiao Yan is a main character who is mentioned to be a young man who is about to be married to the daughter of the Lion Heart General of the Kingdom of Jade. He is also mentioned to have a mysterious background and is described as having a strong personality and a strong sense of justice. Therefore, based on the context, Xiao Yan can be described as a young man who is about to be married to a princess, has a mysterious background, and is known for his strong personality and sense of justice.


In [25]:
res = qa.invoke(
    {"question": "他结婚了吗？"}
)
print(res["answer"])

[WARNING] [05-13 21:56:01] base.py:495 [t:16232]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-13 21:56:01] openapi_requestor.py:316 [t:16232]: requesting llm api endpoint: /chat/eb-instant
[INFO] [05-13 21:56:02] openapi_requestor.py:316 [t:16232]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [05-13 21:56:02] base.py:495 [t:16232]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-13 21:56:02] openapi_requestor.py:316 [t:16232]: requesting llm api endpoint: /chat/eb-instant
[WARNING] [05-13 21:56:05] base.py:495 [t:16232]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-13 21:56:05] openapi_requestor.py:316 [t:16232]: requesting llm api endpoint: /chat/eb-instant


根据原文信息可知，萧炎并没有结婚。文中提到纳兰嫣然对萧炎哥哥与她的婚事最不满意，所以纳兰嫣然此次来解除婚约。因此，可以得出结论，萧炎目前并没有结婚。


In [14]:
#help(QianfanEmbeddingsEndpoint)

* streamlit 的实现 （文件读取问题）

由于和streamlit的文件读取组件不兼容，因此需要通过streamlit读取文件，然后再写入本地，然后再用langchain读取一遍

In [ ]:
def write_text_file(content, file_path):
    try:
        with open(file_path, 'w') as file:
            file.write(content)
        return True
    except Exception as e:
        print(f"Error occurred while writing the file: {e}")
        return False

uploaded_file = st.file_uploader("Upload an article", type="txt")
if uploaded_file is not None:
    content = uploaded_file.read().decode('utf-8')
    # st.write(content)
    file_path = "file.txt"
    write_text_file(content, file_path)   

## 基于pyinstaller的streamlit应用打包

https://blog.csdn.net/Helloorld_1/article/details/131767478

https://blog.csdn.net/Helloorld_1/article/details/132880784

https://blog.csdn.net/nexttoparadise/article/details/135368387